In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_table_experiments as dt
import pandas as pd
import plotly.graph_objs as go
from dash.dependencies import Input, Output, State, Event
import random

In [2]:
##############################################################
        #DATA MANIPULATION (MODEL)
##############################################################
df= pd.read_csv("top500_albums_clean.csv")
df['userscore'] = df['userscore'].astype(float)
df['metascore'] = df['metascore'].astype(float)
df['releasedate']=pd.to_datetime(df['releasedate'], format='%b %d, %Y')
df['year']=df["releasedate"].dt.year
df['decade']=(df["year"]//10)*10
#cleaning Genre
df['genre'] = df['genre'].str.strip()
df['genre'] = df['genre'].str.replace("/", ",")
df['genre'] = df['genre'].str.split(",")
#year trend
df_linechart= df.groupby('year')        .agg({'album':'size', 'metascore':'mean', 'userscore':'mean'})        .sort_values(['year'], ascending=[True]).reset_index()
df_linechart.userscore=df_linechart.userscore*10
#table
df_table= df.groupby('artist').agg({'album':'size', 'metascore':'sum', 'userscore':'sum'})
#genrebubble
df2=(df['genre'].apply(lambda x: pd.Series(x)) .stack().reset_index(level=1, drop=True).to_frame('genre').join(df[['year', 'decade', 'userscore', 'metascore']], how='left') )
df_bubble=  df2.groupby('genre')        .agg({'year':'size', 'metascore':'mean', 'userscore':'mean'})             .sort_values(['year'], ascending=[False]).reset_index().head(15)
df2_decade=df2.groupby(['genre', 'decade']).agg({'year':'size'}) .sort_values(['decade'], ascending=[False]).reset_index()

In [3]:
df_linechart.head()

,year,album,metascore,userscore
0,1999.0,3,87.000000,87.000000
1,2000.0,9,87.888889,86.666667
2,2001.0,24,87.250000,85.583333
3,2002.0,12,87.083333,86.416667
4,2003.0,30,86.666667,86.466667


In [4]:
df_table.head()

,album,metascore,userscore
artist,,,
...And You Will Know Us by the Trail of Dead,1,85.0,8.9
24 Hour Party People,1,86.0,8.5
9th Wonder,1,87.0,7.0
A Tribe Called Quest,1,91.0,8.8
Aesop Rock,1,85.0,8.6


In [5]:
df_bubble.head()

,genre,year,metascore,userscore
0,Rock,358,86.801676,8.303631
1,Pop,207,87.009662,8.144928
2,Alternative,125,86.776000,8.342400
3,Indie Rock,113,86.707965,8.279646
4,Indie,96,86.677083,8.560417


In [6]:
df2_decade.head()

,genre,decade,year
0,Indie Folk,2010.0,9
1,Math Rock,2010.0,1
2,Metalcore,2010.0,1
3,Microsound,2010.0,1
4,Midwest Rap,2010.0,3
